## RKR Computation
All Code and Markdown written by Gary Zeri, Chapman University Student and member of the LaRue Cat Lab

All equations and information within this notebook originated from <i>The Computation of RKR Potential Energy Curves of Diatomic Molecules using Matematica</i>, written by Peter Senn.

The RKR method is a procedure used to determine the potential energy curves of diatomic molecules by calculating the classical turning points, $r_-$ and $r_+$ from the molecule's diatomic constants.


The turning points, $r_{\pm}$, can be computed using the following equation:
$$r_{\pm} = \frac{f(v)}{2} \cdot \left[\sqrt{1 + \frac{1}{f(v)g(v)}} \pm 1\right]$$
<br>
It should be noted that in the above equation the fraction in the square root has a one in the numerator while in the paper by Peter Senn the numerator has a four. The four is not present in the Matematica code in Senn's paper and in <i>Lecture #21: Construction of Potential Curves by the Rydberg-Klein-Rees Method (RKR)</i> from MIT OpenCourseWare, the four is not included as well, leading to the omission of the four in the equation presented here to provide an exact equation to that used within the below code.
<br><br>
$f(v)$ and $g(v)$ are defined as follows: 
$$f(v) = \int^v_{\frac{-1}{2}}{ \left[ \frac{dv^{'}}{\sqrt{E(v)-E(v^{'})}{}} \right] }$$
<br>
$$g(v) = \int_{\frac{-1}{2}}^v{\left[ \frac{\frac{\partial E}{\partial J}}{\sqrt{E(v) - E(v^{'})}}\right]dv^{'}}$$

<br>

In the original paper by Senn, there are constants in front of the integrals that are omitted here as they are used to scale the values to a desired unit system. Since this RKR code will use wavenumbers for energy and angstroms for bond distance, the constant of $\frac{8.211609}{\sqrt{\mu}}$ will be multiplied by the final answers to perform this conversion.
<br>

For vibronic states with no rotation and $J=0$, $E(v)$ and $\frac{\partial E}{\partial J}$ can be represented as:
$$E(v) = \omega_e(v+ \frac{1}{2}) - \omega_ex_e(v+\frac{1}{2})^2 + \omega_ey_e(v+\frac{1}{2})^3 + \omega_ez_e(v+\frac{1}{2})^4 + \dots$$
<br>
$$\frac{\partial E}{\partial J} = B(v) = B_e -\alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2 + \dots$$

<br> 

An important computational issue to note with the RKR method is that the integrand term $\frac{dv^{'}}{\sqrt{E(v) - E(v^{'})}}$ will become one over zero since the integration is from $\frac{-1}{2}$ to $v$, thus the integrand will become $\frac{1}{\sqrt{E(v) - E(v)}}$ when $v^{'} = v$ at the end of the integration.<br>
In order to deal with this issue, the integral is changed from $\int^{v}_{\frac{-1}{2}}$ to $\int^{v-\delta}_{\frac{-1}{2}}$, where $\delta$ is some extremely small value. A correction term is then added to both $f(v)$ and $g(v)$ to account for the missing endpoint of the integration resulting in new and more easily computable versions of $f(v)$ and $g(v)$:

$$f(v) = \int^{v-\delta}_{\frac{-1}{2}}{\frac{dv^{'}}{\sqrt{E(v) - E(v^{'})}} + \sqrt{\frac{\delta}{Q_v}}}$$
<br>
$$g(v) = \int^{v-\delta}_{\frac{-1}{2}}{\frac{B(v^{'})}{\sqrt{E(v)-E(v^{'})}}dv^{'} } + 2B(v)\sqrt{\frac{\delta}{Q_v}} $$
<br>
Where $Q_v$ is the following series:
$$Q_v = w_e-2\omega_ex_e(v+\frac{1}{2}) + 3\omega_ey_e(v+\frac{1}{2})^2 + 4\omega_ez_e(v+\frac{1}{2})^3 + \dots$$

<br>

The following code only uses the terms listed above in the computation for all series listed.

## RKR With Excited States
<br>
In order to allow for RKR computations with excited states, $E(v)$ and $\frac{\partial E}{\partial J}$ must include the $J$ term. The full $E(v)$ equation is included below:
\begin{align}
    E(v) &= \omega_e(v+\frac{1}{2}) - \omega_ex_e(v+\frac{1}{2})^2 + \omega_ey_e(v+\frac{1}{2})^3 + \omega_ez_e(v+\frac{1}{2}) + \left[B_e-\alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2\right] \cdot J(J+1)-D_eJ^2(J+1)^2 \\
    E(v) &= c(v) + \left[B_e-\alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2\right] \cdot J(J+1)-D_eJ^2(J+1)^2 \\
    E(v) &= c(v) + f(v, J) - g(J) 
\end{align}
<br><br>
\begin{align}
    \frac{\partial E}{\partial J} &= \frac{\partial}{\partial J}\left[ c(v) + f(v, J) - g(J)\right] \\
    \frac{\partial E}{\partial J} &= 0 + \frac{\partial}{\partial J}f(v, J) - \frac{\partial}{\partial J}g(J) \\
    \frac{\partial E}{\partial J} &= \frac{\partial}{\partial J}f(v, J) - \frac{\partial}{\partial J}g(J) \\
\end{align}
<br><br>
\begin{align}
    \frac{\partial }{\partial J}f(v, J) &= \frac{\partial }{\partial J}\left[d(v)\cdot h(J)\right] \\
    &= d(v) \frac{\partial}{\partial J}h(v) \\
    &= d(v) \frac{\partial}{\partial J}\left[J(J+1)\right] \\
    &= d(v) \cdot \left(2J + 1\right) \\
    &= \left[B_e-\alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2\right] \cdot \left(2J + 1\right) \\
    &= B_e \cdot \left(2J + 1\right) - \alpha_e(v+\frac{1}{2})\left(2J + 1\right) + y_e(v+\frac{1}{2})^2 \left(2J + 1\right) \\
    &= 2JB_e + B_e -2J\alpha_e(v+\frac{1}{2}) -2\alpha_e(v+\frac{1}{2} ) + 2Jy_e(v+\frac{1}{2})^2 + y_e(v+\frac{1}{2})^2 \\
    &= 2J\left[B_e -\alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2\right] + B_e - \alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2 \\
    &= 2Jd(v) + d(v) \\
    \frac{\partial}{\partial J}f(v, J) &= d(v)\left(2J + 1\right)
\end{align}
<br><br>
\begin{align}
    \frac{\partial }{\partial J}g(J) &= \frac{\partial}{\partial J} \left[D_e J^2(J+1)^2 \right] \\
        &= D_e \frac{\partial}{\partial J}\left[ J^2(J+1)^2\right] \\
        &= D_e \left[ \frac{\partial}{\partial J}[J^2] \cdot (J+1)^2 + J^2 \cdot \frac{\partial}{\partial J}[(J+1)^2]\right] \\
        &= D_e \left[ 2J \cdot (J+1)^2 + J^2 \cdot 2(J+1)\right] \\
        &= 2J(J+1)D_e \left[ (J+1) + J\right] \\
        &= 2J(J+1)D_e\left[ 2J + 1 \right] \\
    \frac{\partial}{\partial J}g(J) &= 2D_eJ(J+1)(2J+1) 
\end{align} 
<br><br>
\begin{align}
        \frac{\partial E}{\partial J} &= \frac{\partial}{\partial J}f(v, J) - \frac{\partial}{\partial J}g(J) \\
       \frac{\partial E}{\partial J} &= d(v)(2J + 1) - 2D_eJ(J+1)(2J+1) \\
       \frac{\partial E}{\partial J} &= \left[B_e -\alpha_e(v+\frac{1}{2}) + y_e(v+\frac{1}{2})^2\right](2J+1) - 2D_eJ(J+1)(2J+1)
\end{align}

In [1]:
#Allow Notebook to Import from Comp_Chem_Package
import sys
sys.path.append("..\\Comp_Chem_Package")

from compChemGlobal import *
from nistScraper import getDiatomicConstants

#Global variables to be modified by the user 

#Distance from v that the integration should stop at
#Should be a very very small number strictly greater than 0
#But, the smaller the value, the slower the integrals will take to compute
delta = pow(10, -3)

resolution = 10

#provide the name of the diatomic constants that are desired
dc = getDiatomicConstants("CO")

In [2]:
#Define All Functions Here

def E(v, J=0):
    term = v + 0.5
    sumValue = (dc["w"] * term) - (dc["wx"]*pow(term, 2)) + (dc["wy"]*pow(term, 3)) + (dc["wz"]*pow(term,4))
    return sumValue * (2*J + 1) - 2*dc["D"]*J*(J+1)*(2*J+1)

def EPrime(v, J=0):
    term = v + 0.5
    return dc["w"] - 2*dc["wx"]*term + 3*dc["wy"]*pow(term, 2) + 4*dc["wz"]*pow(term, 3)

def B(v):
    term = v + 0.5
    return dc["B"] - (dc["a"] * term)  + (dc["y"]*pow(term, 2))

#Used in the correctionFactor calculation
def Q(v):
    term = v + 0.5
    return dc["w"] - (2*dc["wx"]*term) + (3*dc["wy"]*pow(term, 2)) + (4*dc["wz"]*pow(term,3))

#Used to correct integrals that stop delta away from v
def correctionFactor(v):
    return 2 * sqrt(delta / Q(v))

def integralRadical(v, vPrime):
    return sqrt(E(v) - E(vPrime))

def f(v):
    integrand = lambda vPrime: 1 / integralRadical(v, vPrime)
    return integrate(integrand, -0.5, v-delta) + correctionFactor(v)

def g(v):
    integrand = lambda vPrime : B(vPrime) / integralRadical(v, vPrime)
    return integrate(integrand, -0.5, v-delta) + (B(v)*correctionFactor(v))
    
#v refers to a float energy level for the potential well
#returns the tuple (r+, r-)
def RKR(v):
    
    fValue = f(v)
    
    c0 = (8.211609 * fValue ) / (2 * sqrt(dc["u"]))

    radicand = 1 / (fValue * g(v))     
    c1 = sqrt(1 + radicand)
        
    return c0 * (c1 + 1), c0 * (c1 - 1)

In [3]:
#Graphing and Computation Code
import numpy as np

#Lists to hold data 
r = []
EList = []

ddr = []
ddE = []
ddr2 = []
ddE2 = []
leftAsympCutOff = False

v = -0.499
dv = 1 / resolution

loadingBar =  widgets.FloatProgress(
                  value = v, 
                  min = -EPrime(v), 
                  max = 0,
             )
display(widgets.HBox([
              widgets.Label(value="Building RKR Potential"),
              loadingBar
             ]))

#Loop to generate all data to graph
#for v in tqdm(np.arange(vStart, 17, 1/resolution)):
while(EPrime(v) > 0):
    
    energy = E(v)
    rPoints = RKR(v)

    if(not leftAsympCutOff and len(r) != 0):
        #Compute First Derivative
        ddr.append( (rPoints[1] + r[-2:][-1]) / 2 )
        ddE.append( (energy-EList[-1]) / ( rPoints[1] - r[-2:][-1] ))
        
        if(len(ddr) > 1):
            #Compute 2nd Derivative
            ddr2.append( (ddr[-2] + ddr[-1]) / 2 )
            ddE2.append( (ddE[-1] - ddE[-2]) / (ddr[-1] - ddr[-2]) )
            
            #Determine if Cutoff should be used
            if(ddE2[-1] <= 0):
                leftAsympCutOff = True        
    
    if(not leftAsympCutOff):
        r.extend( rPoints )
        EList.extend( [energy] * 2 )
    else:
        r.append(max(rPoints))
        EList.append(energy)
    
    v += dv
    loadingBar.value = -EPrime(v)
    
#Manually add in re point
r.append(dc["re"])
EList.append(0)

#Prepare data for graphing
figure = {
    "data":[
        {
            "type":"scatter",
            "x":r,
            "y":EList,
            "connectgaps":True,
            "mode":"markers",
            "name":"RKR Potential",
        }
    ],
    
    "layout":{
        "xaxis":{"title":"r in Angstroms"},
        "yaxis":{"title":"Energy in Wavenumbers"},
        "title":{"text":"RKR Generated Potential Energy Surface"}
    }
}

display(plot.go.FigureWidget(data=figure))

FigureWidget({
    'data': [{'connectgaps': True,
              'mode': 'markers',
              'name': 'RKR …

In [1]:
#Test for the RKR Class
#Allow Notebook to Import from Comp_Chem_Package
import sys
sys.path.append("..\\Comp_Chem_Package")
from rkr import rkr as rkrClass

from compChemGlobal import *
from nistScraper import getDiatomicConstants

dc = getDiatomicConstants("CO")

rkr = rkrClass(dc)
rkr.compute(resolution=50, delta = pow(10, -3))
print("Graphing Potential")
rkr.graph()

Graphing Potential


FigureWidget({
    'data': [{'hoverlabel': {'font': {'size': 16}},
              'hovertemplate': '<b>r in Ang…